# ETL and Feature Creation

This script will import the already 'cleaned' dataset I prepared for last submission.

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [17]:
#Import cleaned dataset. It is the one that was exported with the script from last submission. 
df = pd.read_csv("US_Accidents_June20_CLEAN_Week1.csv")
#the csv file was saved with the index column, so I'll drop it
df = df.drop(columns=['Unnamed: 0'])
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Duration,Start_Lat,Start_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,0 days 05:14:00.000000000,39.865147,-84.058723,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,0 days 00:30:00.000000000,39.928059,-82.831184,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,0 days 00:30:00.000000000,39.063148,-84.032608,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,0 days 00:30:00.000000000,39.747753,-84.205582,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,0 days 00:30:00.000000000,39.627781,-84.188354,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [18]:
#Inspect the columns
print(df.columns)

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time', 'Duration',
       'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Description', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset',
       'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'],
      dtype='object')


## ETL

In terms of ETL, since there is only one dataset, I don't have to care about the merge with other data.<br>
I'll do a sanity check of all types present in the dataset and transform the ones I consider.

In [19]:
#Import dataset with description, feature types and examples that I created for the last submission.
df_typecheck = pd.read_csv('Week1_Variable_Summary_Table.csv')
df_typecheck = df_typecheck.loc[df_typecheck.Attribute.isin(df.columns)]
df_typecheck = df_typecheck.drop(columns=['Unnamed: 0', '#'])
df_typecheck.reset_index( inplace=True)
df_typecheck = df_typecheck.drop(columns=['index', 'Description', 'Example'])
df_typecheck

,Attribute,Data type
0,ID,<class 'str'>
1,Source,<class 'str'>
2,TMC,<class 'numpy.float64'>
3,Severity,<class 'numpy.int64'>
4,Start_Time,<class 'pandas._libs.tslibs.timestamps.Timesta...
5,End_Time,<class 'pandas._libs.tslibs.timestamps.Timesta...
6,Start_Lat,<class 'numpy.float64'>
7,Start_Lng,<class 'numpy.float64'>
8,Distance(mi),<class 'numpy.float64'>
9,Description,<class 'str'>


This table is apparently wrong, since when exported to csv the data type Timestamp apparently changes to String.
I'll change the datatype for 'Start_Time', 'End_Time' and 'Weather_Timestamp': timestamps.Timestamp.<br>
Also, I'm missing the feature I created called 'Duration', since it was created after I created the table.

In [20]:
#Change all times to timestamps.Timestamp-data type.
df['Start_Time'] = pd.to_datetime(df.Start_Time, format='%Y-%m-%d %H:%M:%S')
df['End_Time'] = pd.to_datetime(df.End_Time, format='%Y-%m-%d %H:%M:%S')
df['Weather_Timestamp'] = pd.to_datetime(df.Weather_Timestamp, format='%Y-%m-%d %H:%M:%S')

Now that we have all the Timestamps in the same format, let's calculate the difference between last weather record and the time of the accident in order to see how up to date the weather records are.

In [21]:
#Calculate difference between Start_Time and Weather_Timestamp

weather_time_diff = df.Weather_Timestamp - df.Start_Time
#Transform to seconds
weather_time_diff = weather_time_diff.dt.total_seconds()
#Filter out biggest differences
weather_time_diff = weather_time_diff.sort_values(ascending=True)

#See how many of the Weather_Timestamps are more than one day away from the time of the accident. 
weather_bool_big_diff_vec = (weather_time_diff > 86400)|(weather_time_diff < -86400)
print('Number of Weather_Timestamps with more than 1 day difference wrt accident time:', weather_bool_big_diff_vec.sum())

Number of Weather_Timestamps with more than 1 day difference wrt accident time: 0


All of the Weather_Timestamps are within one day from the accident time, which is good. That means the Weather_Conditions are up to date and reflect the real weather at the time of the accident.

### Deal with Sunset/Twilight

There are 4 Features that describe if at the time of the accident it was 'Day' or 'Night'. Since in my opinion the influence of this factor is mainly in the vision of the driver. I've read through the 4 descriptions and decided I'll only look at the Sunrise_Sunset feature, since it is the one that describes when the sun is in line with the horizont.

In [22]:
df = df.drop(columns=['Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'])
df.columns.shape

(42,)

### Imputting Missing Data

For the moment I'll only impute the float values with the median of that particular feature (more robust towards outliers than mean). For String features, a more detailed analysis should be done.

In [23]:
# I'll impute using the median value for that column.
#For computational efficiency reasons, I'll create a list with only the Features where NaN-values are present. So, I won't iterate through all Features to impute the NaNs.
nan_vec = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']

for element in nan_vec:
    df[element].fillna(df[element].median(), inplace=True)
    
print()
print(df.isnull().sum()/(df.shape[0])*100)

BEFORE 64.0
AFTER 64.0
BEFORE 67.0
AFTER 67.0
BEFORE 29.95
AFTER 29.95
BEFORE 10.0
AFTER 10.0
BEFORE 7.0
AFTER 7.0

ID                    0.000000
Source                0.000000
TMC                  29.451104
Severity              0.000000
Start_Time            0.000000
End_Time              0.000000
Duration              0.000000
Start_Lat             0.000000
Start_Lng             0.000000
Distance(mi)          0.000000
Description           0.000028
Street                0.000000
Side                  0.000000
City                  0.003188
County                0.000000
State                 0.000000
Zipcode               0.030424
Country               0.000000
Timezone              0.110428
Airport_Code          0.192337
Weather_Timestamp     1.233003
Temperature(F)        0.000000
Humidity(%)           0.000000
Pressure(in)          0.000000
Visibility(mi)        0.000000
Wind_Direction        1.675595
Wind_Speed(mph)       0.000000
Weather_Condition     2.166941
Amenity         

## Feature Engineering

In [9]:
# Create a shorter list for 'Weather_Condition'

print('', df['Weather_Condition'].unique())

 ['Light Rain' 'Overcast' 'Mostly Cloudy' 'Rain' 'Light Snow' 'Haze'
 'Scattered Clouds' 'Partly Cloudy' 'Clear' 'Snow'
 'Light Freezing Drizzle' 'Light Drizzle' 'Fog' 'Shallow Fog' 'Heavy Rain'
 'Light Freezing Rain' 'Cloudy' 'Drizzle' nan 'Light Rain Showers' 'Mist'
 'Smoke' 'Patches of Fog' 'Light Freezing Fog' 'Light Haze'
 'Light Thunderstorms and Rain' 'Thunderstorms and Rain' 'Fair'
 'Volcanic Ash' 'Blowing Sand' 'Blowing Dust / Windy' 'Widespread Dust'
 'Fair / Windy' 'Rain Showers' 'Mostly Cloudy / Windy'
 'Light Rain / Windy' 'Hail' 'Heavy Drizzle' 'Showers in the Vicinity'
 'Thunderstorm' 'Light Rain Shower' 'Light Rain with Thunder'
 'Partly Cloudy / Windy' 'Thunder in the Vicinity' 'T-Storm'
 'Heavy Thunderstorms and Rain' 'Thunder' 'Heavy T-Storm' 'Funnel Cloud'
 'Heavy T-Storm / Windy' 'Blowing Snow' 'Light Thunderstorms and Snow'
 'Heavy Snow' 'Low Drifting Snow' 'Light Ice Pellets' 'Ice Pellets'
 'Squalls' 'N/A Precipitation' 'Cloudy / Windy' 'Light Fog' 'Sand'
 'Snow 

In [19]:
df_rain = df[df['Weather_Condition'].str.contains("Rain", na = False)]
df_rain['Weather_Condition'].unique()

array(['Light Rain', 'Rain', 'Heavy Rain', 'Light Freezing Rain',
       'Light Rain Showers', 'Light Thunderstorms and Rain',
       'Thunderstorms and Rain', 'Rain Showers', 'Light Rain / Windy',
       'Light Rain Shower', 'Light Rain with Thunder',
       'Heavy Thunderstorms and Rain', 'Rain / Windy',
       'Heavy Rain / Windy', 'Heavy Freezing Rain', 'Heavy Rain Showers',
       'Rain Shower', 'Light Rain Shower / Windy',
       'Light Freezing Rain / Windy', 'Freezing Rain', 'Rain and Sleet',
       'Freezing Rain / Windy', 'Heavy Rain Shower'], dtype=object)

In [17]:
df_rain['Weather_Condition']

268640           Snow Grains
784502     Light Snow Grains
784504     Light Snow Grains
784505     Light Snow Grains
2080873    Light Snow Grains
2080874    Light Snow Grains
2341837    Light Snow Grains
2494605          Snow Grains
2494608          Snow Grains
2494609          Snow Grains
Name: Weather_Condition, dtype: object